In [ ]:
# !pip install pandas_gbq
# !pip install pydata_google_auth
# !pip install google-cloud-bigquery-storage

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(r'C:\Users\kevin.puika\OneDrive - Bank Jasa Jakarta\Desktop\KRP-CKPN\joey-bi-ss-risk-fraud-project-36e37a4eccd7 1.json')
project_id = "joey-bi-ss-risk-fraud-project"
client = bigquery.Client(credentials= credentials,project=project_id)


In [2]:
## Dont forget to Update Asset Selling & Asset Inventory Table to Risk-fraud-project ...

## Asset Selling:

# drop table joey-bi-ss-risk-fraud-project.jf.JF_ASSETSELLING_v_2;
# create table joey-bi-ss-risk-fraud-project.jf.JF_ASSETSELLING_v_2 as
# SELECT * FROM joey-bi-prod-project.staging.stg_jf__asset_selling


## Asset 

# drop table joey-bi-ss-risk-fraud-project.jf.JF_ASSETINVENTORY_v_2;
# create table joey-bi-ss-risk-fraud-project.jf.JF_ASSETINVENTORY_v_2 as
# SELECT * FROM joey-bi-prod-project.staging.stg_jf__asset_inventory


In [2]:
sql = '''
SELECT AGRMNT_NO, SELLING_DT, SOLD_PRICE 
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY AGRMNT_NO ORDER BY GENERATED_DT ASC) AS row_number 
    FROM `joey-bi-ss-risk-fraud-project.jf.JF_ASSETSELLING_v_2`
) t 
WHERE t.row_number = 1
'''
reposold = client.query(sql).to_dataframe()
#reposold.to_excel(r'C:\Data\JF\CKPN\2024only\reposold.xlsx')
reposold

,AGRMNT_NO,SELLING_DT,SOLD_PRICE
0,0777502417758,09/09/2024,3813696.00
1,0777VEHICLE02300485,30/11/2023,118067766.00
2,0777VEHICLE02301121,28/06/2024,75304206.00
3,0777VEHICLE02302417,31/05/2024,104093259.00
4,0777VEHICLE02303172,16/08/2024,91761515.00
...,...,...,...
1368,0777VEHICLE02405786,23/11/2024,126873700.00
1369,0777VEHICLE02406358,25/02/2025,169480171.00
1370,0777VEHICLE02410498,01/07/2024,104282660.00
1371,0777VEHICLE02414419,31/08/2025,40263871.00


In [3]:
sql = '''
SELECT AGRMNT_NO, REPO_DT, INVENTORY_DT, CUSTOMER_NAME, OS_PRINCIPLE_BANK 
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY AGRMNT_NO ORDER BY GENERATED_DT ASC) AS row_number 
    FROM `joey-bi-ss-risk-fraud-project.jf.JF_ASSETINVENTORY_v_2`
) t 
WHERE t.row_number = 1
'''
invn = client.query(sql).to_dataframe()

invn

,AGRMNT_NO,REPO_DT,INVENTORY_DT,CUSTOMER_NAME,OS_PRINCIPLE_BANK
0,0777502306932,29/08/2024,29/08/2024,HENNY SUSANTI PURBA,13070727.00
1,0777VEHICLE02300388,22/03/2024,22/03/2024,MUHAMAD RUSLI,119365949.00
2,0777VEHICLE02300575,01/07/2024,30/07/2024,ROMDONI,122784344.00
3,0777VEHICLE02301465,21/05/2025,21/05/2025,HUSEN,117403994.00
4,0777VEHICLE02302051,14/05/2025,14/05/2025,HARDIYANTI A,119902957.00
...,...,...,...,...,...
1428,0777VEHICLE02410580,06/08/2024,26/08/2024,LUSI SEPTARIA,144343900.00
1429,0777VEHICLE02413846,21/05/2025,21/05/2025,ISMAIL,99968847.00
1430,0777VEHICLE02414422,19/06/2024,20/06/2024,RIYANTO,129272360.00
1431,0777VEHICLE02414519,30/06/2025,30/06/2025,EVY ROSVIANTY,137617758.00


In [4]:
import pandas as pd

repomerge = pd.merge(invn, reposold,  how='left', left_on=['AGRMNT_NO'], right_on = ['AGRMNT_NO'])
repomerge['OS_PRINCIPLE_BANK'] = pd.to_numeric(repomerge['OS_PRINCIPLE_BANK'], errors='coerce')
repomerge['SOLD_PRICE'] = pd.to_numeric(repomerge['SOLD_PRICE'], errors='coerce')
repomerge['GAIN_LOSS'] = repomerge['SOLD_PRICE'] - repomerge['OS_PRINCIPLE_BANK']
repomerge

,AGRMNT_NO,REPO_DT,INVENTORY_DT,CUSTOMER_NAME,OS_PRINCIPLE_BANK,SELLING_DT,SOLD_PRICE,GAIN_LOSS
0,0777502306932,29/08/2024,29/08/2024,HENNY SUSANTI PURBA,13070727.0,25/09/2024,8246939.0,-4823788.0
1,0777VEHICLE02300388,22/03/2024,22/03/2024,MUHAMAD RUSLI,119365949.0,18/04/2024,84711368.0,-34654581.0
2,0777VEHICLE02300575,01/07/2024,30/07/2024,ROMDONI,122784344.0,30/07/2024,77446324.0,-45338020.0
3,0777VEHICLE02301465,21/05/2025,21/05/2025,HUSEN,117403994.0,30/07/2025,68733770.0,-48670224.0
4,0777VEHICLE02302051,14/05/2025,14/05/2025,HARDIYANTI A,119902957.0,16/06/2025,78476978.0,-41425979.0
...,...,...,...,...,...,...,...,...
1428,0777VEHICLE02410580,06/08/2024,26/08/2024,LUSI SEPTARIA,144343900.0,20/09/2024,84182980.0,-60160920.0
1429,0777VEHICLE02413846,21/05/2025,21/05/2025,ISMAIL,99968847.0,28/06/2025,85815915.0,-14152932.0
1430,0777VEHICLE02414422,19/06/2024,20/06/2024,RIYANTO,129272360.0,29/07/2024,88031414.0,-41240946.0
1431,0777VEHICLE02414519,30/06/2025,30/06/2025,EVY ROSVIANTY,137617758.0,29/08/2025,86364014.0,-51253744.0


In [5]:
repomerge.to_pickle(r'C:\Users\kevin.puika\OneDrive - Bank Jasa Jakarta\Desktop\Porto Data\Agunan JF\newrepo2.tar.gz')

In [6]:
repomerge.to_excel(r'C:\Users\kevin.puika\OneDrive - Bank Jasa Jakarta\Desktop\Porto Data\Agunan JF\newrepo2.xlsx')

import pandas as pd
#tambel sementara
repo_csv = pd.read_csv (r'C:\Users\kevin.puika\OneDrive - Bank Jasa Jakarta\Desktop\repo_csv.csv')

repo_csv.to_pickle(r'C:\Users\kevin.puika\OneDrive - Bank Jasa Jakarta\Desktop\Porto Data\Agunan JF\newrepo2.tar.gz')
repo_csv.to_excel(r'C:\Users\kevin.puika\OneDrive - Bank Jasa Jakarta\Desktop\Porto Data\Agunan JF\newrepo2.xlsx')